# Stock Analysis Agent – local setup\n\nThis notebook lets another user spin up the stock-analysis web app on their own Windows machine by running cells from top to bottom.\n\n**What this notebook does:**\n- Sets the project directory.\n- Installs Python dependencies from `requirements.txt`.\n- Guides you through installing and configuring Ollama (one-time manual step).\n- Starts the FastAPI backend with Uvicorn.\n- Opens the web UI in a browser at `http://localhost:8000/index.html`.\n\n> You must have Python 3.10+ installed. Ollama installation itself cannot be automated from this notebook, but we provide the exact commands to run in a separate terminal.

In [ ]:
import os, pathlib\n\n# === 1. Set your project folder path here ===\n# Change this to match your own username and folder if needed.\nPROJECT_DIR = pathlib.Path(r"c:\\Users\\first_last\\Documents\\Project Folder").resolve()\nprint("Using project directory:", PROJECT_DIR)\n\nif not PROJECT_DIR.exists():\n    raise FileNotFoundError(f"Project directory does not exist: {PROJECT_DIR}")\n\nos.chdir(PROJECT_DIR)\nprint("Current working directory:", pathlib.Path.cwd())

In [ ]:
import subprocess, sys, pathlib\n\n# === 2. Install Python dependencies ===\nreq_path = pathlib.Path("requirements.txt")\nif not req_path.exists():\n    raise FileNotFoundError("requirements.txt not found in project directory")\n\nprint("Installing dependencies from", req_path)\nsubprocess.check_call([sys.executable, "-m", "pip", "install", "-r", str(req_path)])\nprint("\nAll dependencies installed.")

## 3. Install and configure Ollama (one-time manual step)\n\nRun these commands **once** in a separate terminal (PowerShell or Command Prompt), not inside the notebook:\n\n1. Install Ollama for Windows from the official website.\n2. Open a terminal and pull a model (example using `llama3.1`):\n   ```powershell\n   ollama pull llama3.1\n   ```\n3. Make sure the Ollama server is running (usually starts automatically):\n   ```powershell\n   ollama serve\n   ```\n\nOnce Ollama is installed and serving, you can verify from this notebook:

In [ ]:
import subprocess\n\nprint("Checking that Ollama is installed and responding...")\ntry:\n    result = subprocess.run(["ollama", "list"], capture_output=True, text=True, check=True)\n    print(result.stdout)\nexcept FileNotFoundError:\n    raise RuntimeError("`ollama` command not found. Please install Ollama and ensure it is on your PATH.")\nexcept subprocess.CalledProcessError as e:\n    print("Ollama command returned an error:")\n    print(e.stderr or e.stdout)

In [ ]:
import subprocess, os, sys\n\n# === 4. Start FastAPI with Uvicorn ===\n# Configure environment variables for the LLM backend.\nos.environ["LLM_BACKEND"] = "ollama"\nos.environ["LLM_MODEL_NAME"] = os.environ.get("LLM_MODEL_NAME", "llama3.1")\n\nprint("Starting Uvicorn server on http://127.0.0.1:8000 ...")\nuvicorn_cmd = [sys.executable, "-m", "uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]\n\n# Start in the background so the notebook remains usable.\nuvicorn_proc = subprocess.Popen(uvicorn_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)\nprint("Uvicorn PID:", uvicorn_proc.pid)\nprint("\nYou can view logs by running `uvicorn_proc.stdout.read()` in a separate cell if needed.")

In [ ]:
import webbrowser, time\n\n# === 5. Open the UI in your default browser ===\ntime.sleep(3)  # give the server a moment to start\nurl = "http://localhost:8000/index.html"\nprint("Opening", url)\nwebbrowser.open(url)

In [ ]:
# (Optional) Stop the Uvicorn server started from this notebook.\ntry:\n    uvicorn_proc.terminate()\n    print("Terminated Uvicorn process", uvicorn_proc.pid)\nexcept NameError:\n    print("No Uvicorn process found in this notebook session.")